In [15]:
n0 = 14800
n1 = n0 + 243

In [1]:
import pandas as pd
import os
import numpy as np
import networkx as nx

##### import data

In [21]:
authorship = pd.read_csv('/Users/yanmemgxing/Nutstore Files/Nutstore/Chemistry/author_paper_year.csv')
paper_reference = pd.read_csv(f'/Users/yanmemgxing/Nutstore Files/Nutstore/Chemistry/paper_reference_20.csv')
paper_auCount = pd.read_csv(f'/Users/yanmemgxing/Nutstore Files/Nutstore/Chemistry/paper_auCount.csv')

In [10]:
author_id_info = pd.read_csv('author_id_info.csv')
neuro = pd.read_csv('chemistry.csv')

In [11]:
author_id_info_mte = author_id_info.rename(columns={'AFTAuthorID':'MenteeID','author_id':'mentee_id','careerlen':'mentee_careerlen',
                                                    'pubsnum':'mentee_pubsnum'})
neuro = neuro.merge(author_id_info_mte,on='MenteeID',how='left')


author_id_info_mto = author_id_info.rename(columns={'AFTAuthorID':'MentorID','author_id':'mentor_id','careerlen':'mentor_careerlen',
                                                    'pubsnum':'mentor_pubsnum'})
neuro = neuro.merge(author_id_info_mto,on='MentorID',how='left')

In [12]:
neuro.head(5)

,Unnamed: 0,CID,MenteeID,MentorID,MentorshipType,Institution,InstitutionMAGID,StartYear,StopYear,MAGAuthorID_x,mentee_id,mentee_careerlen,mentee_pubsnum,MAGAuthorID_y,mentor_id,mentor_careerlen,mentor_pubsnum
0,3,6,18761,9,1,"University of California, Berkeley",95457486.0,-1,1984,2.809701e+09,A2809701389,37.0,278.0,2.952982e+09,A2952981685,46.0,92.0
1,117,130,53604,33687,1,Utrecht University,193662353.0,-1,1997,1.874842e+09,A1874842480,27.0,109.0,NaN,NaN,NaN,NaN
2,137,151,20,24,2,Baylor College of Medicine,181547552.0,-1,-1,2.169128e+09,A2169128453,37.0,70.0,2.225823e+09,A2225823178,43.0,161.0
3,147,161,12561,52639,1,"University of Washington, Seattle",201448701.0,1967,1971,2.307650e+09,A2307649786,39.0,131.0,2.619948e+09,A2619947921,18.0,18.0
4,190,212,188,189,1,Harvard University,136199984.0,-1,1927,2.039886e+09,A2039885679,46.0,18.0,2.152986e+09,A2152986058,116.0,43.0


In [13]:
neuro = neuro[(neuro.mentee_pubsnum>=20)&(neuro.mentor_pubsnum>=20)]

In [14]:
len(neuro)

51113

In [37]:
len(paper_reference)

56234220

In [12]:
savepath = f'/Users/yanmemgxing/Nutstore Files/Nutstore/Neuro/Community/' 

In [13]:
def network_community(mte_case,mto_case,authorship,paper_reference,paper_auCount,savepath):
    Pe = list(authorship['work_id'][authorship.author_id==mte_case].drop_duplicates())
    Po = list(authorship['work_id'][authorship.author_id==mto_case].drop_duplicates())
    Pe_Po = list(set(Pe)&set(Po))

    # print('len(Pe):',len(Pe))
    # print('len(Po):',len(Po))

    # node_list
    Pe_df = pd.DataFrame(Pe,columns=['Id'])
    Pe_df['Label'] = 'e'

    Po_df = pd.DataFrame(Po,columns=['Id'])
    Po_df['Label'] = 'o'

    node_list = pd.concat((Pe_df,Po_df)).reset_index().drop(columns=['index'])

    index_common = list(node_list[node_list.Id.isin(Pe_Po)].index)
    node_list.loc[index_common,'Label'] = 'e&o'
    node_list = node_list.drop_duplicates()

    # edge_list
    C = paper_reference[['work_id','referenced_work_id']][paper_reference.referenced_work_id.isin(node_list.Id)]
    C_copy = C.copy()
    C_cocited = C.merge(C_copy,on='work_id',how='left')
    C_cocited = C_cocited[['referenced_work_id_x','referenced_work_id_y']][(~C_cocited.referenced_work_id_x.isnull())
                                       &(~C_cocited.referenced_work_id_y.isnull())
                                      &(C_cocited.referenced_work_id_x<C_cocited.referenced_work_id_y)]
    edge_list = C_cocited.groupby(['referenced_work_id_x','referenced_work_id_y']).size().reset_index(name='Weight')
    edge_list = edge_list.rename(columns={'referenced_work_id_x':'Source','referenced_work_id_y':'Target'})
    
    # save edge_list
    newpath = f'{savepath}{mte_case}_{mto_case}/'
    if not os.path.exists(newpath):
        os.makedirs(newpath)

    #save file
    edge_list.to_csv(newpath+'edge_list.csv',index=0)
    
    # create network
    G = nx.Graph()
    G.add_nodes_from(node_list.Id.tolist())
    G.add_weighted_edges_from(edge_list.values.tolist())
    try:
        communities = nx.community.greedy_modularity_communities(G,weight='weight',resolution=0.8)
        competition_communities = []
        community_n = 0
        # community_label = []
        for k in communities:
            k_nodes = list(k)
            # add community columns
            index_community = list(node_list[node_list.Id.isin(k_nodes)].index)
            node_list.loc[index_community,'Community'] = community_n            # add community label to node_list

            # print(len(k_nodes))
            k_pe = [node for node in k_nodes if node in Pe]
            k_po = [node for node in k_nodes if node in Po]
            e,o = len(k_pe)/len(k_nodes),len(k_po)/len(k_nodes)
            # print(community_n,len(k_nodes),e,o)
            if len(k_nodes)>=10:    #e>=0.15 and o>=0.15 and 
                competition_communities.append((community_n,k_nodes))
                # print(community_n,len(k_nodes),e,o)

            community_n+=1

        #save node_list
        node_list.to_csv(newpath+'node_list.csv',index=0) 
    except:
        competition_communities = []
        
        #save node_list
        node_list.to_csv(newpath+'node_list.csv',index=0)
    
    
    
    if len(competition_communities)>0:
        for community_k in competition_communities:
            topic_i = community_k[0]
            community_i = community_k[1]
            cc = community_i
            Citation_cc = paper_reference[['work_id','referenced_work_id']][paper_reference.referenced_work_id.isin(cc)]
            Citing_set = Citation_cc.groupby('work_id').size().reset_index(name='Count')
            Cociting_set = Citing_set['work_id'][Citing_set.Count>1]
            Cited_weight = Citation_cc[Citation_cc.work_id.isin(Cociting_set)].groupby('referenced_work_id').size().reset_index(name='Weight').rename(columns={'referenced_work_id':'work_id'})
            CS = Cited_weight.merge(paper_auCount, how='left', on='work_id')
            CS['weight_frac_credit'] = CS.Weight/CS.author_counts
            node_list = node_list.rename(columns={'Id':'work_id'})
            CS = CS.merge(node_list, how='left', on='work_id')
            CS = CS.merge(authorship[['work_id','publication_year']], how='left', on='work_id').drop_duplicates().sort_values(by='publication_year')
            # Credit_mentee = sum(CS.weight_frac_credit[(CS.Label=='e')|(CS.Label=='e&o')])
            # Credit_mentor = sum(CS.weight_frac_credit[(CS.Label=='o')|(CS.Label=='e&o')])
            k_pe = [node for node in cc if node in Pe]
            k_po = [node for node in cc if node in Po]
            e,o = len(k_pe)/len(cc),len(k_po)/len(cc)
            
            #save community and credit : topic_i,  e, o
            newpath = f'{savepath}{mte_case}_{mto_case}/topic/'
            if not os.path.exists(newpath):
                os.makedirs(newpath)
                
            CS.to_csv(f'{newpath}{topic_i}_{e}_{o}.csv',index=0)
            
    else:
        a = 'nt'
    
    return print('saved successfully')

In [ ]:
mte_mto = neuro[['mentee_id','mentor_id']].values.tolist()
count = 0
# n0=0
# n1=15
for k in mte_mto[n0:n1]:
    mte_case = k[0]
    mto_case = k[1]
    network_community(mte_case,mto_case,authorship,paper_reference,paper_auCount,savepath)
    count+=1
    if count%10==0:
        print(count)

saved successfully
saved successfully
saved successfully
saved successfully
saved successfully
saved successfully
saved successfully
saved successfully
